# COS597N Project Notebook

In [5]:
import pickle
import os
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss, accuracy_score, matthews_corrcoef
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
import re
import tqdm
import torch.nn.functional as F
from datetime import datetime
from copy import deepcopy
from scipy.special import softmax

/home/byw2/.conda/envs/dr-bert-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df_train = pickle.load(open('../Datasets/train.pkl', "rb"))
df_val = pickle.load(open('../Datasets/val.pkl', "rb"))
df_test = pickle.load(open('../Datasets/caid.pkl', "rb"))

In [7]:
print(f'Training Dataset: {df_train.shape}')
print(f'Validation Dataset: {df_val.shape}')
print(f'Test Dataset: {df_test.shape}')

Training Dataset: (1569, 4)
Validation Dataset: (176, 4)
Test Dataset: (652, 3)


In [8]:
652/(1569+176+652)

0.2720066750104297

In [9]:
df_train.columns

Index(['Sequence', 'full', 'disprot_ID', 'intersect'], dtype='object')

In [10]:
df_train.head()

,Sequence,full,disprot_ID,intersect
0,MELITNELLYKTYKQKPVGVEEPVYDQAGDPLFGERGAVHPQSTLK...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00675,False
5,MEDINFASLAPRHGSRPFMGNWQDIGTSNMSGGAFSWGSLWSGIKN...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00808,False
7,MAAKFEVGSVYTGKVTGLQAYGAFVALDEETQGLVHISEVTHGFVK...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00809,False
9,VVYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00137,False
11,MNYIPTQTFYGRRWRPRPAARPWPLQATPVAPVVPDFQAQQMQQLI...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00999,False


In [11]:
df_val.head()

,Sequence,full,disprot_ID,intersect
1823,MIPVTILCVLLCLNLAWAQDGKTTFEKEGGGGRGPRILENMHESSC...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00233,False
1283,MGNLESTDGGPGEPPSVPLLLPPGKTPMPEPCELEERFALVLSSMN...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP02638,False
1868,MSGSVSGCGSGGCSIVWFRRDLRVEDNPALAAAVRAGPVIALFVWA...,"[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",DP00474,False
1416,MATVNQLVRKPRARKVAKSNVPALEACPQKRGVCTRVYTTTPKKPN...,"[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",DP00145,False
170,MAQWEMLQNLDSPFQDQLHQLYSHSLLPVDIRQYLAVWIEDQNWQE...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",DP00961,False


In [12]:
df_test.head()

,ID,Sequence,full
0,DP00084,MSDNDDIEVESDEEQPRFQSAADKRAHHNALERKRRDHIKDSFHSL...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DP00182,MAPTKRKGSCPGAAPKKPKEPVQVPKLVIKGGIEVLGVKTGVDSFT...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,DP00206,MKAAQKGFTLIELMIVVAIIGILAAIAIPAYQDYTARAQLSERMTL...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DP00334,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DP00359,MMLTKSVVISRPAVRPVSTRRAVVVRASGQPAVDLNKKVQDAVKEA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
df_full = pd.concat([df_train, df_val, df_test])

In [14]:
df_full.iloc[0]['Sequence']

'MELITNELLYKTYKQKPVGVEEPVYDQAGDPLFGERGAVHPQSTLKLPHKRGERDVPTNLASLPKRGDCRSGNSRGPVSGIYLKPGPLFYQDYKGPVYHRAPLELFEEGSMCETTKRIGRVTGSDGKLYHIYVCIDGCIIIKSATRSYQRVFRWVHNRLDCPLWVTTCSDTKEEGATKKKTQKPDRLERGKMKIVPKESEKDSKTKPPDATIVVEGVKYQVRKKGKTKSKNTQDGLYHNKNKPQESRKKLEKALLAWAIIAIVLFQVTMGENITQWNLQDNGTEGIQRAMFQRGVNRSLHGIWPEKICTGVPSHLATDIELKTIHGMMDASEKTNYTCCRLQRHEWNKHGWCNWYNIEPWILVMNRTQANLTEGQPPRECAVTCRYDRASDLNVVTQARDSPTPLTGCKKGKNFSFAGILMRGPCNFEIAASDVLFKEHERISMFQDTTLYLVDGLTNSLEGARQGTAKLTTWLGKQLGILGKKLENKSKTWFGAYAASPYCDVDRKIGYIWYTKNCTPACLPKNTKIVGPGKFGTNAEDGKILHEMGGHLSEVLLLSLVVLSDFAPETASVMYLILHFSIPQSHVDVMDCDKTQLNLTVELTTAEVIPGSVWNLGKYVCIRPNWWPYETTVVLAFEEVSQVVKLVLRALRDLTRIWNAATTTAFLVCLVKIVRGQMVQGILWLLLITGVQGHLDCKPEFSYAIAKDERIGQLGAEGLTTTWKEYSPGMKLEDTMVIAWCEDGKLMYLQRCTRETRYLAILHTRALPTSVVFKKLFDGRKQEDVVEMNDNFEFGLCPCDAKPIVRGKFNTTLLNGPAFQMVCPIGWTGTVSCTSFNMDTLATTVVRTYRRSKPFPHRQGCITQKNLGEDLHNCILGGNWTCVPGDQLLYKGGSIESCKWCGYQFKESEGLPHYPIGKCKLENETGYRLVDSTSCNREGVAIVPQGTLKCKIGKTTVQVIAMDTKLGPMPCRPYEIISSEGPVEKTACTFNYTKTLKNKY

In [15]:
protein_lens = pd.Series([len(seq) for seq in df_full['Sequence']])
protein_lens.describe()

count     2397.000000
mean       549.033792
std        884.678317
min         19.000000
25%        204.000000
50%        392.000000
75%        651.000000
max      34350.000000
dtype: float64

Great, we can verify that none of the training and validation examples overlap with the test example.

In [16]:
MAX_LENGTH = 1024
# EPOCHS = 10
EPOCHS = 1
LEARNING_RATE = 2e-6
BATCH_SIZE = 1
TOKENIZER_PATH =  "../checkpoint-final/"
# is this pretrained on protein sequences?
PRETRAINED_MODEL = "../checkpoint-final/"
NUM_CLASSES = 2
SCHEDULER='cosine_with_restarts'

In [17]:
class ProteinDegreeDataset(Dataset):

    def __init__(self, max_length, df, tokenizer, region_type):
        self.region_type = region_type # e.g. 'full'
        self.df = df
        self.seqs, self.labels = self.load_dataset()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def load_dataset(self):
        seq = list(self.df['Sequence']) # list of protein sequences
        label = list(self.df[self.region_type]) # list of list of labels
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)

        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        tens = torch.tensor(self.labels[idx], dtype=torch.long)
        sample['labels'] = F.pad(tens, (0, MAX_LENGTH - len(tens)))
        return sample

In [18]:
tokenizer = RobertaTokenizerFast.from_pretrained(TOKENIZER_PATH, do_lower_case=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
train_dataset = ProteinDegreeDataset(MAX_LENGTH, df_train, tokenizer, 'full')
# train_dataset = ProteinDegreeDataset(MAX_LENGTH, df_train., tokenizer, 'full')
val_dataset = ProteinDegreeDataset(MAX_LENGTH, df_val, tokenizer, 'full')
# val_dataset = ProteinDegreeDataset(MAX_LENGTH, df_val, tokenizer, 'full')

In [140]:
len(val_dataset)

176

In [141]:
train_dataset[0]

{'input_ids': tensor([ 4, 21,  9,  ..., 14,  5,  0]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'labels': tensor([0, 0, 0,  ..., 0, 0, 0])}

In [142]:
pd.Series(train_dataset[0]['input_ids']).value_counts()

5     93
15    83
12    83
7     82
8     67
9     63
16    53
11    52
13    51
6     48
10    47
14    45
17    41
18    40
20    39
23    37
19    31
21    23
24    23
22    21
4      1
0      1
dtype: int64

There are 25 tokens in the vocabulary (20 amino acids and ["unk_token": "[UNK]", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"]).

In [143]:
pd.Series(train_dataset[0]['attention_mask']).value_counts()

1    1024
dtype: int64

All tokens should be attended to.

In [144]:
print(len(train_dataset[0]['attention_mask']), len(train_dataset[0]['input_ids']), len(train_dataset[0]['labels']))

1024 1024 1024


Finally, note that all training points are truncated to be 1024 amino acids for training.

In [152]:
def precision_recall_f1_roc_convolve(name, logits, labels, convolution):
    convolved = np.convolve(np.array(logits).flatten(), np.array(convolution / np.sum(convolution)).flatten(), 'same')
    p = [(1 - i, i) for i in convolved]
    roc = [i[1] for i in p]
    roc2 = [i[0] for i in p]
    p = np.argmax(p, axis=-1)
    precision, recall, f1, support = precision_recall_fscore_support(labels, p)
    roc_auc = roc_auc_score(labels, roc)
    mcc = matthews_corrcoef(labels, p)
    return {
        f'precision_{name}':precision[1],
        f'recall_{name}':recall[1],
        f'f1_{name}':f1[1],
        f'roc_auc_{name}':roc_auc,
        f'mcc_{name}': mcc,
    }

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    # creates probabilities
    logits = softmax(logits, axis=2)
    l = []
    # concatenates all true labels for sequence into list
    for j, i in enumerate(labels):
        l = l + list(i[:len(df_val['Sequence'].iloc[j])])
    # concatenates all prob that were predicted to be disordered into list
    lg2 = []
    for k, i in enumerate(logits):
        lg2 = lg2 + [j[1] for j in i[:len(df_val['Sequence'].iloc[k])]]
    print(np.array(lg2).shape)
    # l is list of all concatenated true labels
    # lg2 is list of all concatenated probabilities that corresponding residue in l is equal to 1  
    metrics = {}
    metrics.update(precision_recall_f1_roc_convolve('normal', lg2, l, [1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa5', lg2, l, [1,1,1,1,1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa9', lg2, l, [1,1,1,1,1,1,1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa15', lg2, l, [1]*15))
#     metrics.update(precision_recall_f1_roc_convolve('linear5', lg2, l, [1,2,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('linear9', lg2, l, [1,2,3,4,5,4,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('linear15', lg2, l, [1,2,3,4,5,6,7,8,7,6,5,4,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad5', lg2, l, [1,3,9,3,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad9', lg2, l, [1,3,9,27,81,27,9,3,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad15', lg2, l, [1,3,9,27,81,243,729,2187,729,243,81,27,9,3,1]))
    
    logits_path = OUTPUT_DIR + '/Logits/'
    if not os.path.isdir(logits_path):
        os.mkdir(logits_path)
    new_df = deepcopy(df_val)
    new_df['Logits'] = [[i[1] for i in x] for x in list(logits)]
    pickle.dump(new_df, open(logits_path + datetime.now().strftime("%H:%M:%S"), 'wb'))
    return metrics

In [162]:
a = [0, 0, 1, 1, 0]
b = [0.3, 0.2, 0.5, 0.9, 0.1]
tmp = np.convolve(np.array(b).flatten(), np.array([1, 1] / np.sum([1, 1])).flatten(), 'same')
print(tmp)

[0.15 0.25 0.35 0.7  0.5 ]


In [154]:
OUTPUT_DIR = f'../Outputs'

In [155]:
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR + '/Checkpoints',
    num_train_epochs = EPOCHS, # 10 epochs
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 16,
    warmup_steps = 1000,
    learning_rate = LEARNING_RATE, # 2e-06
    logging_dir = OUTPUT_DIR + '/Logs',
    logging_steps = 200,
    lr_scheduler_type=SCHEDULER, # cosine with restarts
    do_train = True,
    do_eval = True,
    evaluation_strategy = 'epoch', # evaluate at every epoch
    gradient_accumulation_steps = BATCH_SIZE,
#     fp16 = True,
#     fp16_opt_level = '02',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [156]:
from transformers import AutoModelForTokenClassification
def model_init():
    model = AutoModelForTokenClassification.from_pretrained(PRETRAINED_MODEL, num_labels=NUM_CLASSES)
    return model

In [157]:
trainer = Trainer(
    model_init=model_init,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [158]:
trainer.train()

/home/byw2/.conda/envs/dr-bert-env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision Normal,Recall Normal,F1 Normal,Roc Auc Normal,Mcc Normal
1,0.310700,0.259302,0.693860,0.200245,0.310796,0.794237,0.309684


(86238,)


TrainOutput(global_step=785, training_loss=0.34079548052162123, metrics={'train_runtime': 42.5016, 'train_samples_per_second': 36.916, 'train_steps_per_second': 18.47, 'total_flos': 409989427900416.0, 'train_loss': 0.34079548052162123, 'epoch': 1.0})

In [30]:
!conda list torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
# packages in environment at /home/byw2/.conda/envs/dr-bert-env:
#
# Name                    Version                   Build  Channel
pytorch                   2.0.1           cpu_py39hab5cca8_0  
